In [1]:
%reload_ext autoreload
%autoreload 2

import argparse
import os
import sys
import numpy as np
from timeit import default_timer as timer
from _collections import OrderedDict
import shutil
from pprint import pprint
start = timer()
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.sqlcontroller import SqlController
from utilities.file_location import FileLocationManager
from utilities.utilities_cvat_neuroglancer import get_structure_number, NumpyToNeuroglancer, get_segment_properties
from utilities.utilities_affine import rigid_transform_3D, ralign, affine_fit

Connecting dklab@192.168.1.12:3306


In [3]:
animal = 'MD589'
fileLocationManager = FileLocationManager(animal)
atlas_name = 'atlasV7'
DATA_PATH = '/net/birdstore/Active_Atlas_Data/data_root'
ROOT_DIR = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data'
THUMBNAIL_DIR = os.path.join(ROOT_DIR, animal, 'preps', 'CH1', 'thumbnail')
ATLAS_PATH = os.path.join(DATA_PATH, 'atlas_data', atlas_name)
ORIGIN_PATH = os.path.join(ATLAS_PATH, 'origin')
VOLUME_PATH = os.path.join(ATLAS_PATH, 'structure')
OUTPUT_DIR = os.path.join(fileLocationManager.neuroglancer_data, 'atlas')
#if os.path.exists(OUTPUT_DIR):
#    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)
origin_files = sorted(os.listdir(ORIGIN_PATH))
volume_files = sorted(os.listdir(VOLUME_PATH))
sqlController = SqlController(animal)
resolution = sqlController.scan_run.resolution
surface_threshold = 0.8
SCALE = (10 / 0.46)

structure_volume_origin = {}
for volume_filename, origin_filename in zip(volume_files, origin_files):
    structure = os.path.splitext(volume_filename)[0]
    if structure not in origin_filename:
        print(structure, origin_filename)
        break

    color = get_structure_number(structure.replace('_L', '').replace('_R', ''))

    origin = np.loadtxt(os.path.join(ORIGIN_PATH, origin_filename))
    volume = np.load(os.path.join(VOLUME_PATH, volume_filename))

    volume = np.rot90(volume, axes=(0, 1))
    volume = np.flip(volume, axis=0)
    volume[volume > surface_threshold] = color
    volume = volume.astype(np.uint8)

    structure_volume_origin[structure] = (volume, origin)

col_length = sqlController.scan_run.width/SCALE
row_length = sqlController.scan_run.height/SCALE
z_length = len(os.listdir(THUMBNAIL_DIR))
atlasV7_volume = np.zeros(( int(row_length), int(col_length), z_length), dtype=np.uint8)
print('atlas volume shape', atlasV7_volume.shape)

DK52_centers = {'12N': [46488, 18778, 242],
                '5N_L': [38990, 20019, 172],
                '5N_R': [39184, 19027, 315],
                '7N_L': [42425, 23190, 166],
                '7N_R': [42286, 22901, 291]}
##### actual data for both sets of points, pixel coordinates
MD589_centers = {'10N_L': [31002.069009677187, 17139.273764067697, 210],
                 '10N_R': [30851.821452912456, 17026.27799914138, 242],
                 '4N_L': [25238.351916435207, 13605.972626040299, 210],
                 '4N_R': [25231.77274616, 13572.152382002621, 236],
                 '5N_L': [25863.93885802854, 16448.49802904827, 160],
                 '5N_R': [25617.920248719453, 16089.048882550318, 298],
                 '7N_L': [27315.217906796195, 18976.4921239128, 174],
                 '7N_R': [27227.134448911638, 18547.6538128018, 296],
                 '7n_L': [26920.538205417844, 16996.292850204114, 177],
                 '7n_R': [26803.347723222105, 16688.23325135847, 284],
                 'Amb_L': [29042.974021303286, 18890.218579368557, 167],
                 'Amb_R': [28901.503217056554, 18291.072163747285, 296],
                 'DC_L': [28764.5378815116, 15560.1247992853, 134],
                 'DC_R': [28519.240424058273, 14960.063579837733, 330],
                 'LC_L': [26993.749068166835, 15146.987356709138, 180],
                 'LC_R': [26951.610128773387, 14929.363532303963, 268],
                 'Pn_L': [23019.18002537938, 17948.490571838032, 200],
                 'Pn_R': [23067.16403704933, 17945.89008778571, 270],
                 'SC': [24976.373217129738, 10136.880464106176, 220],
                 'Tz_L': [25210.29041867189, 18857.20817842522, 212],
                 'Tz_R': [25142.520897455783, 18757.457820947686, 262]}
centers = OrderedDict(MD589_centers)
centers_list = []
for value in centers.values():
    centers_list.append((value[1]/SCALE, value[0]/SCALE, value[2]))
COM = np.array(centers_list)
atlas_com_centers = OrderedDict()
atlas_all_centers = {}
for structure, (volume, origin) in sorted(structure_volume_origin.items()):
    midcol, midrow, midz = origin
    row_start = midrow + row_length / 2
    col_start = midcol + col_length / 2
    z_start = midz / 2 + z_length / 2
    row_end = row_start + volume.shape[0]
    col_end = col_start + volume.shape[1]
    z_end = z_start + (volume.shape[2] + 1) / 2
    midcol = (col_end + col_start) / 2
    midrow = (row_end + row_start) / 2
    midz = (z_end + z_start) / 2
    if structure in centers.keys():
        atlas_com_centers[structure] = [midrow, midcol, midz]
    atlas_all_centers[structure] = [midrow, midcol, midz]
ATLAS_centers = OrderedDict(atlas_com_centers)
ATLAS = np.array(list(ATLAS_centers.values()))

atlas volume shape (1490, 2010, 447)


In [24]:
#####Steps
trn = affine_fit(ATLAS, COM)
R, t = rigid_transform_3D(ATLAS.T, COM.T)
#R, t = ralign(ATLAS, COM)
pprint(R)
t = np.reshape(t, (1,3))
pprint(t.shape)

array([[ 0.99669058,  0.04718899, -0.06618979],
       [-0.04692206,  0.99888295,  0.0055824 ],
       [ 0.06637928, -0.00245817,  0.99779144]])
(1, 3)


In [26]:
for structure, (volume, origin) in sorted(structure_volume_origin.items()):
    print(str(structure).ljust(6),end=": ")


    ## generic atlas
    x,y,z = origin
    arr = np.array([y,x,z])
    input = arr + t
    results = np.dot(R, input.T)
    #print(origin, end="\t")
    #print(results)
    
    #y = results[0]
    #x = results[1]
    #z = results[2]
    y = results[0][0]
    x = results[1][0]
    z = results[2][0]
    x_start = int( round(x + col_length / 2))
    y_start = int( round(y + row_length / 2))
    z_start = int(z) // 2 + z_length // 2
    x_end = x_start + volume.shape[0]
    y_end = y_start + volume.shape[1]
    z_end = z_start + (volume.shape[2] + 1) // 2

    print('X',
          str(x_start).rjust(4),
          str(x_end).rjust(4),
          'Y',
          str(y_start).rjust(4),
          str(y_end).rjust(4),
          'Z',
          str(z_start).rjust(4),
          str(z_end).rjust(4),
          end=" ")


    if structure in centers.keys():
        midy = (y_end + y_start) / 2
        midx = (x_end + x_start) / 2
        midz = (z_end + z_start) / 2

        xo,yo,zo = MD589_centers[structure]
        print('COM ERR',
              round(midy*SCALE-yo, 2),
              round(midx*SCALE-xo, 2),
              round(midz - zo, 2),
              end=" ")
    z_indices = [z for z in range(volume.shape[2]) if z % 2 == 0]
    volume = volume[:, :, z_indices]
    volume = np.swapaxes(volume, 0, 1)

    try:
        #atlasV7_volume[row_start:row_end,col_start:col_end, sec_start:sec_end] += volume
        atlasV7_volume[y_start:y_end, x_start:x_end, z_start:z_end] += volume
    except ValueError as ve:
        print(ve, end=" ")

    print()

# rotation puts in view with wide x and shallow  y
atlasV7_volume = np.rot90(atlasV7_volume, 1)
# fliplr and flip axis=1 did not work, axis=0 works
atlasV7_volume = np.flip(atlasV7_volume, axis=0)
print('Done')

10N_L : X 1430 1560 Y  740  828 Z  172  199 COM ERR -95.8 1497.93 -24.5 operands could not be broadcast together with shapes (88,60,27) (88,130,27) (88,60,27)  
10N_R : X 1430 1560 Y  737  825 Z  201  228 COM ERR -48.02 1648.18 -27.5 operands could not be broadcast together with shapes (88,60,27) (88,130,27) (88,60,27)  
12N   : X 1424 1548 Y  751  829 Z  178  221 operands could not be broadcast together with shapes (78,66,43) (78,124,43) (78,66,43)  
3N_L  : X 1124 1187 Y  579  620 Z  176  202 
3N_R  : X 1124 1187 Y  577  618 Z  189  215 
4N_L  : X 1179 1198 Y  595  618 Z  179  190 COM ERR -421.19 598.6 -25.5 
4N_R  : X 1180 1199 Y  591  614 Z  203  214 COM ERR -474.33 626.92 -27.5 
5N_L  : X 1203 1276 Y  677  763 Z  115  145 COM ERR -796.32 1081.71 -30.0 
5N_R  : X 1204 1277 Y  659  745 Z  252  282 COM ERR -828.18 1349.47 -31.0 
6N_L  : X 1286 1312 Y  714  739 Z  175  188 
6N_R  : X 1287 1313 Y  709  734 Z  211  224 
7N_L  : X 1260 1361 Y  802  878 Z  121  168 COM ERR -715.62 1173.91

In [ ]:
%%time
ng = NumpyToNeuroglancer(atlasV7_volume, [10000, 10000, 20000])
ng.preview()
ng.init_precomputed(OUTPUT_DIR)
ng.add_segment_properties(get_segment_properties())
ng.add_downsampled_volumes()
ng.add_segmentation_mesh()